In [1]:
!pip install datasets huggingface_hub torch torchvision validators transformers fvcore bitsandbytes wandb

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 80.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 93.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 93.4 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 93.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 87.6 MB/s eta 0:00:00
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61399 sha2

In [3]:
!pip install gigagan-pytorch==0.2.20

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 83.1 MB/s eta 0:00:00
  Created wheel for numerize: filename=numerize-0.12-py3-none-any.whl size=3154 sha256=2bdd6a1dcad8bf79e9b1a7c44edffb4c5c34415af849872500979403240c9bd0
  Stored in directory: /root/.cache/pip/wheels/6a/49/3e/5375462d832133c3684a27f9ad763a61141a802aee4bd445d6
Successfully built numerize
wandb: WARNING Using legacy-service, which is deprecated. If this is unintentional, you can fix it by ensuring you do not call `wandb.require('legacy-service')` and do not set the WANDB_X_REQUIRE_LEGACY_SERVICE environment variable.
wandb: Logging into wandb.ai. (Learn how to deploy a 

In [6]:
import wandb
#Replace with actual key
wandb.login(key='')

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sb9855 (sb9855-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [7]:
import torch
import torch.nn as nn
import os
from torch.cuda.amp import autocast # Automatic Mixed Precision
import bitsandbytes as bnb # For 8-bit quantization
import torch.profiler # For profiling
from torchvision.utils import save_image # To save images (local saving optional)
import traceback # For detailed error messages
import wandb # Weights & Biases

# Attempt to import GigaGAN, will be mocked if not available for basic structure
try:
    from gigagan_pytorch import GigaGAN, Discriminator # Assuming Discriminator might be needed for type hints or explicit construction
except ImportError:
    print("Warning: gigagan_pytorch library not found. Using a mock GigaGAN class for demonstration purposes.")
    # Define a mock GigaGAN if the library isn't available
    class GigaGAN(nn.Module):
        def __init__(self, generator, discriminator, amp=False, train_upsampler=False, **kwargs): # Added discriminator to match expected signature
            super().__init__()
            g_config = generator if generator else {}
            d_config = discriminator if discriminator else {} # Use discriminator config
            s_config = g_config.get('style_network', {})

            self.G = nn.Module() 
            self.G.style_network = nn.Sequential(nn.Linear(s_config.get('dim', 64), s_config.get('dim', 64)))
            self.G.dim = g_config.get('dim', 32)
            self.G.image_size = g_config.get('image_size', 256)
            self.G.input_image_size = g_config.get('input_image_size', 64)
            self.G.unconditional = g_config.get('unconditional', True)
            self.G.unet = nn.Module() # Mock unet
            self.G.unet.allowable_rgb_resolutions = [self.G.input_image_size] # Mock property

            self.G.conv1 = nn.Conv2d(3, self.G.dim, kernel_size=3, padding=1)
            self.G.relu1 = nn.ReLU()
            dummy_conv_out_size = self.G.dim * (self.G.input_image_size // 2)**2 if self.G.input_image_size > 0 and self.G.dim > 0 else 2048
            self.G.linear_large = nn.Linear(max(1, dummy_conv_out_size), max(1, dummy_conv_out_size) * 2)
            nn.init.normal_(self.G.linear_large.weight, mean=0, std=0.01)
            if self.G.linear_large.bias is not None: nn.init.zeros_(self.G.linear_large.bias)

            self.G.final_conv = nn.Conv2d(self.G.dim if self.G.dim > 0 else 3, 3, kernel_size=3, padding=1)
            self.G.sigmoid = nn.Sigmoid()
            self.G.attention_block = nn.MultiheadAttention(embed_dim=max(1,self.G.dim), num_heads=1) if self.G.dim > 0 else nn.Identity()
            self.G.norm_layer = nn.LayerNorm(max(1,self.G.dim)) if self.G.dim > 0 else nn.Identity()
            self.G.style_processor = nn.Linear(max(1,self.G.dim), max(1,self.G.dim)) if self.G.dim > 0 else nn.Identity()
            self.G.mapping_network = nn.Linear(max(1,self.G.dim), max(1,self.G.dim)) if self.G.dim > 0 else nn.Identity()
            
            self.amp = amp # Store amp
            self.device = torch.device("cpu")

        def load(self, checkpoint_path):
            print(f"MockGigaGAN: Simulating loading weights from {checkpoint_path}")
            return self

        def generate(self, lowres_image=None, batch_size=1, *args, **kwargs):
            current_batch_size = lowres_image.shape[0] if lowres_image is not None else batch_size
            final_image_size = self.G.image_size
            if lowres_image is None: 
                lowres_image = torch.randn(current_batch_size, 3, self.G.input_image_size, self.G.input_image_size, device=self.device)
                if self.amp and self.device.type == 'cuda': lowres_image = lowres_image.half()
            dtype = torch.float16 if self.amp and self.device.type == 'cuda' else torch.float32
            generated_image = torch.rand(current_batch_size, 3, final_image_size, final_image_size, device=self.device, dtype=dtype)
            return generated_image

        def to(self, *args, **kwargs):
            super().to(*args, **kwargs)
            try: self.device = next(self.parameters()).device
            except StopIteration:
                found_device = None
                for arg in args:
                    if isinstance(arg, (str, torch.device)): found_device = torch.device(arg); break
                if found_device: self.device = found_device
                elif 'device' in kwargs: self.device = torch.device(kwargs['device'])
            if hasattr(self, 'G'): self.G.to(self.device)
            return self
    
    # Define a dummy Discriminator if the real one is not available for type hints
    if 'Discriminator' not in globals():
        class Discriminator(nn.Module):
            def __init__(self, **kwargs):
                super().__init__()
                print(f"Mock Discriminator initialized with {kwargs}")
                self.fc = nn.Linear(10,1) # Dummy layer

In [8]:
class GigaGANQuantizedV100(GigaGAN):
    def __init__(self, *args, **kwargs):
        self.amp = kwargs.get('amp', False) 
        super().__init__(*args, **kwargs)
        # self.device is typically set by .to(device) call after instantiation
        # or inherited if superclass sets it based on parameters.

    def quantize_for_v100(self):
        print("Starting quantization for V100 GPU...")
        if not hasattr(self, 'G') or self.G is None:
            raise AttributeError("Generator 'G' is not initialized. Cannot quantize.")
        
        if not hasattr(self, 'device') or self.device is None: # Ensure device is known
            try: self.device = next(self.G.parameters()).device
            except StopIteration: self.device = torch.device("cpu"); print("Warning: G has no params, device defaulted to CPU for quantize.")
        print(f"Generator 'G' target device for quantization: {self.device}")

        if not list(self.G.parameters()):
             print("Warning: Generator 'G' has no parameters. Quantization steps might not be effective.")
        else:
            print(f"Generator 'G' current device: {next(self.G.parameters()).device}")

        if self.device.type == 'cuda':
            self.G.half()
            print("Model G converted to FP16.")
        else:
            print("Skipping .half() conversion as model is not on CUDA.")

        self._preserve_critical_components(self.G)
        print("Critical components preservation step complete.")

        if self.device.type == 'cuda': 
            self._selective_int8_conversion()
            print("Selective INT8 quantization step complete.")
        else:
            print("Skipping INT8 conversion as model is not on CUDA.")

        self._add_stability_hooks()
        print("Stability hooks step complete.")
        print("Quantization complete!")

    def _preserve_critical_components(self, module):
        for name, child in module.named_children():
            if any(x in name.lower() for x in ['attention', 'norm', 'style', 'mapping']):
                child.float()
                for param in child.parameters(): param.data = param.data.float()
                self._preserve_critical_components(child) 
            elif len(list(child.children())) > 0:
                self._preserve_critical_components(child)

    def _selective_int8_conversion(self):
        total_converted = 0
        if not hasattr(self, 'G') or not (hasattr(self, 'device') and self.device.type == 'cuda'):
            print("Skipping INT8: G not found or not on CUDA device.")
            return
        if not list(self.G.parameters()) or not next(self.G.parameters()).is_cuda:
             print("Warning: G is not on CUDA or has no parameters. Skipping bnb INT8 conversion.")
             return

        for name, module_instance in self.G.named_modules():
            if (isinstance(module_instance, nn.Linear) and
                hasattr(module_instance, 'weight') and module_instance.weight is not None and
                module_instance.weight.numel() > 1_000_000 and # Example threshold
                not any(x in name.lower() for x in ['attention', 'style', 'mapping', 'norm'])):
                name_parts = name.rsplit('.', 1)
                parent_name_str = name_parts[0] if len(name_parts) > 1 else ''
                layer_attr_name = name_parts[1] if len(name_parts) > 1 else name
                parent_module = self.G
                if parent_name_str:
                    try: parent_module = self.G.get_submodule(parent_name_str)
                    except AttributeError: print(f"Warn: Parent submod '{parent_name_str}' for '{name}' not found. Skip INT8."); continue
                if not hasattr(parent_module, layer_attr_name): print(f"Warn: Layer '{layer_attr_name}' for '{name}' not in parent. Skip INT8."); continue
                
                original_layer = getattr(parent_module, layer_attr_name)
                if isinstance(original_layer, bnb.nn.Linear8bitLt): continue
                try:
                    new_linear = bnb.nn.Linear8bitLt(
                        original_layer.in_features, original_layer.out_features,
                        bias=original_layer.bias is not None, has_fp16_weights=True, threshold=6.0)
                    fp16_weight_data = original_layer.weight.data.clone().to(self.device, dtype=torch.float16)
                    new_linear.weight = bnb.nn.Int8Params(fp16_weight_data, requires_grad=False, has_fp16_weights=True)
                    if original_layer.bias is not None:
                        new_linear.bias = original_layer.bias.data.clone().to(self.device, dtype=torch.float16)
                    setattr(parent_module, layer_attr_name, new_linear.to(self.device))
                    total_converted += 1
                except Exception as e: print(f"Failed to convert {name} to INT8: {e}")
        if total_converted > 0: print(f"Total layers converted to INT8: {total_converted}")
        else: print("No eligible linear layers found or converted to INT8.")

    def _add_stability_hooks(self):
        if not hasattr(self, 'G'): return
        def pre_forward_hook(module, input_args):
            if hasattr(module, 'parameters') and len(list(module.parameters())) > 0 and \
               next(module.parameters()).is_cuda and next(module.parameters()).dtype == torch.float16:
                if isinstance(input_args[0], torch.Tensor) and input_args[0].dtype == torch.float32:
                     return (input_args[0].half(),) + input_args[1:] if len(input_args) > 1 else (input_args[0].half(),)
            return input_args
        for name, module_instance in self.G.named_modules(): 
            if isinstance(module_instance, nn.Conv2d) and len(list(module_instance.parameters())) > 0 and next(module_instance.parameters()).is_cuda:
                module_instance.register_forward_pre_hook(pre_forward_hook)

    def generate(self, *args, **kwargs):
        if not hasattr(self, 'device') or self.device is None:
             try: self.device = next(self.G.parameters()).device if hasattr(self, 'G') and list(self.G.parameters()) else torch.device("cpu")
             except StopIteration: self.device = torch.device("cpu"); print("Warn: Device for generate() defaulted to CPU.")
        
        for key_to_check in ['noise', 'lowres_image']:
            if key_to_check in kwargs and kwargs[key_to_check] is not None:
                tensor_input = kwargs[key_to_check]
                if isinstance(tensor_input, torch.Tensor):
                    target_dtype = torch.float16 if self.amp and self.device.type == 'cuda' else tensor_input.dtype
                    kwargs[key_to_check] = tensor_input.to(self.device, dtype=target_dtype)
        autocast_enabled = self.amp and self.device.type == 'cuda'
        with autocast(enabled=autocast_enabled):
            with torch.inference_mode():
                if 'truncation_psi' not in kwargs and hasattr(self.G, 'truncation_psi'): kwargs['truncation_psi'] = 0.7
                generated = super().generate(*args, **kwargs)
                if isinstance(generated, torch.Tensor): generated = generated.float()
                return generated

In [18]:
def load_quantized_gigagan(checkpoint_path, device_str='cuda', unconditional_gan=True, image_size=256, input_image_size=64):
    current_device = torch.device(device_str)
    print(f"Initializing GigaGANQuantizedV100 on device: {current_device}")
    is_upsampler = input_image_size < image_size
    
    # This is a critical assumption for multiscale_input_resolutions.
    # If the real GigaGAN has a way to report allowable resolutions (e.g., via unet), that would be more robust.
    # For now, based on past errors, it seems tied to the generator's input_image_size.
    current_multiscale_resolution = [input_image_size] 
    print(f"Setting D multiscale_input_resolutions to: {current_multiscale_resolution}")

    gan = GigaGANQuantizedV100(
        train_upsampler=True,  # Assuming upsampler model
        generator=dict(
            style_network=dict(
                dim=64,
                depth=4
            ),
            dim=32,
            image_size=256,
            input_image_size=64,
            unconditional=True
        ),
        discriminator=dict(
            dim_capacity=16,
            dim_max=512,
            image_size=256,
            num_skip_layers_excite=4,
            multiscale_input_resolutions=(128,),
            unconditional=True
        ),
        amp=True
    ).to('cuda') # Move to device
    
    # Ensure self.device is set on gan instance after .to()
    #gan.device = current_device 
    if hasattr(gan.G, 'parameters') and len(list(gan.G.parameters())) > 0 :
        print(f"Model G device after .to(): {next(gan.G.parameters()).device}")
    else:
        print("Model G has no parameters or not fully initialized for device check after .to()")


    if os.path.exists(checkpoint_path):
        try:
            gan.load(checkpoint_path) 
            print(f"Successfully called gan.load({checkpoint_path})")
        except Exception as e:
            print(f"Warning: gan.load() failed for {checkpoint_path}: {e}. Using initialized weights.")
    else:
        print(f"Warning: Checkpoint {checkpoint_path} not found. Using initialized weights.")

    gan.quantize_for_v100() 
    gan.eval()
    print("Model is in evaluation mode.")
    return gan


In [19]:
def generate_images_and_profile(gan, num_images=5, output_dir="generated_images_quantized_profiler", seed=42, profile_log_dir='./profiler_logs', export_chrome_trace=True):
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(profile_log_dir, exist_ok=True) 
    torch.manual_seed(seed)
    
    if not hasattr(gan, 'device') or gan.device is None: # Should be set by load_quantized_gigagan
        gan.device = next(gan.parameters()).device if list(gan.parameters()) else torch.device("cpu")
    print(f"Generating images on device: {gan.device}")

    input_h = gan.G.input_image_size if hasattr(gan.G, 'input_image_size') else 64
    input_w = gan.G.input_image_size if hasattr(gan.G, 'input_image_size') else 64
    is_upsampler_model = hasattr(gan.G, 'input_image_size') and gan.G.input_image_size < gan.G.image_size

    example_input_tensor = None; generate_kwargs = {}
    if is_upsampler_model or not gan.G.unconditional:
        print(f"Model type: Upsampler/Conditional. Dummy input size: ({input_h}, {input_w}).")
        example_input_tensor = torch.randn(1, 3, input_h, input_w, device=gan.device)
        generate_kwargs['lowres_image'] = example_input_tensor
    else: 
        print(f"Model type: Unconditional from-scratch. generate_kwargs['batch_size']=1.")
        generate_kwargs['batch_size'] = 1
    if example_input_tensor is not None and gan.amp and gan.device.type == 'cuda':
        example_input_tensor = example_input_tensor.half()
        if 'lowres_image' in generate_kwargs: generate_kwargs['lowres_image'] = example_input_tensor
    if example_input_tensor is not None: print(f"Dummy input tensor: shape {example_input_tensor.shape}, dtype {example_input_tensor.dtype}")
    
    profiler_schedule = torch.profiler.schedule(wait=1, warmup=1, active=max(1, num_images-2), repeat=1)
    print(f"\nStarting profiling session (wait=1, warmup=1, active={max(1, num_images-2)})...")
    profiler_activities = [torch.profiler.ProfilerActivity.CPU]
    if gan.device.type == 'cuda': profiler_activities.append(torch.profiler.ProfilerActivity.CUDA)

    with torch.profiler.profile(
        activities=profiler_activities, schedule=profiler_schedule, record_shapes=True,
        profile_memory=True, with_stack=True, with_flops=True
    ) as prof:
        for i in range(num_images):
            with torch.no_grad(): image = gan.generate(**generate_kwargs)
            if i < 3 and wandb.run: # Log a few sample images to wandb
                try:
                    image_cpu_wandb = image.float().cpu().clamp(0.,1.)
                    wandb.log({f"generated_sample_{i}": wandb.Image(image_cpu_wandb)}, step=wandb.run.step) # Use global step if available or i
                except Exception as e_wandb_img:
                    print(f"Warning: Could not log image {i} to wandb: {e_wandb_img}")
            if prof: prof.step()
    print(f"\nFinished generating {num_images} images (local saving commented out during profiling).")
    
    if prof and wandb.run: # Ensure profiler ran and wandb is active
        print("\n--- PyTorch Profiler Results (logged to wandb) ---")
        key_averages = prof.key_averages()
        sort_key = "self_cpu_time_total"; has_cuda_events = False
        if gan.device.type == 'cuda':
            for event in key_averages:
                if hasattr(event, 'self_cuda_time_total') and event.self_cuda_time_total > 0: has_cuda_events = True; break
            if has_cuda_events: sort_key = "self_cuda_time_total"
            else: print("No significant CUDA activity in profiler, sorting by CPU time for console.")
        
        # Log profiler table to wandb
        try:
            table_data = []; columns = ["Operator", "Self CPU %", "Self CPU", "CPU total %", "CPU total", "Calls"]
            if has_cuda_events: columns.extend(["Self CUDA %", "Self CUDA", "CUDA total %", "CUDA total"])
            
            total_cpu_time = sum(evt.cpu_time for evt in key_averages) # Or use prof.total_average().cpu_time
            total_cuda_time = sum(evt.cuda_time for evt in key_averages if hasattr(evt, 'cuda_time')) if has_cuda_events else 0

            for ev_avg in sorted(key_averages, key=lambda x: getattr(x, sort_key, 0), reverse=True):
                row = [
                    ev_avg.key,
                    f"{ev_avg.self_cpu_time_percent:.2f}%", f"{ev_avg.self_cpu_time_total/1000:.2f}ms",
                    f"{ev_avg.cpu_time_percent:.2f}%", f"{ev_avg.cpu_time_total/1000:.2f}ms",
                    ev_avg.count
                ]
                if has_cuda_events:
                    row.extend([
                        f"{getattr(ev_avg, 'self_cuda_time_percent', 0.0):.2f}%", f"{getattr(ev_avg, 'self_cuda_time_total', 0)/1000:.2f}ms",
                        f"{getattr(ev_avg, 'cuda_time_percent', 0.0):.2f}%", f"{getattr(ev_avg, 'cuda_time_total', 0)/1000:.2f}ms"
                    ])
                table_data.append(row)
            if table_data:
                profiler_wandb_table = wandb.Table(columns=columns, data=table_data)
                wandb.log({"pytorch_profiler_summary": profiler_wandb_table})
                print("Logged PyTorch Profiler summary as a wandb.Table.")
        except Exception as e: print(f"Error creating/logging wandb.Table for profiler: {e}")
        
        print(f"Console Profiler Table (sorted by {sort_key}):")
        print(key_averages.table(sort_by=sort_key, row_limit=15))

        if export_chrome_trace:
            trace_file_name = f"gigagan_trace_{gan.device.type}_{wandb.run.id}.json"
            trace_file_path = os.path.join(profile_log_dir, trace_file_name)
            try:
                prof.export_chrome_trace(trace_file_path)
                print(f"\nChrome trace file saved to: {trace_file_path}")
                profile_artifact = wandb.Artifact(name=f"profiler-trace-{wandb.run.id}", type="profile")
                profile_artifact.add_file(trace_file_path)
                wandb.log_artifact(profile_artifact)
                print(f"Logged Chrome trace artifact to wandb: {profile_artifact.name}")
            except Exception as e: print(f"\nCould not export or log Chrome trace: {e}")
    elif prof: print("\n--- PyTorch Profiler Results (wandb run not active, printing to console only) ---") ; print(prof.key_averages().table(row_limit=15))
    else: print("Profiler was not active or did not record data.")

In [20]:
USE_CUDA = torch.cuda.is_available()
DEVICE_STR = "cuda" if USE_CUDA else "cpu" 
DEVICE = torch.device(DEVICE_STR)

DUMMY_CHECKPOINT_PATH = "./model-32.ckpt" # Matching user's previous log
if not os.path.exists(DUMMY_CHECKPOINT_PATH):
    print(f"Creating dummy checkpoint: {DUMMY_CHECKPOINT_PATH}")
    torch.save({"generator": {}, "discriminator": {}}, DUMMY_CHECKPOINT_PATH)

# --- Parameters ---
CHECKPOINT_TO_LOAD = DUMMY_CHECKPOINT_PATH
NUM_IMAGES_TO_GENERATE = 5 
OUTPUT_IMAGE_DIR = "generated_output_wandb" # Local output if any
PROFILE_LOG_DIR = './profiler_wandb_logs' 
EXPORT_CHROME_TRACE_FILE = True 

IMAGE_SIZE = 64 # Final image size
INPUT_IMAGE_SIZE = 32 # Input to upsampler / base size for profiler error
UNCONDITIONAL_GAN = True

# --- Initialize W&B Run ---
run_name = f"GigaGAN_QProf_in{INPUT_IMAGE_SIZE}_out{IMAGE_SIZE}_{DEVICE_STR}_{wandb.util.generate_id()[:4]}"
try:
    wandb.init(
        project="gigagan-profiling-experiments", # REPLACE with your project name
        name=run_name,
        config={
            "checkpoint": CHECKPOINT_TO_LOAD, "num_images": NUM_IMAGES_TO_GENERATE,
            "image_size": IMAGE_SIZE, "input_image_size": INPUT_IMAGE_SIZE,
            "unconditional_gan": UNCONDITIONAL_GAN, "device": DEVICE_STR,
            "quantization_target": "V100_style_mixed_precision", "profiling_enabled": True,
            "export_chrome_trace": EXPORT_CHROME_TRACE_FILE
        },
        # mode="offline" # Use if you don't have internet access during run
    )
    print(f"Wandb run initialized: {wandb.run.name if wandb.run else 'FAILED'} (ID: {wandb.run.id if wandb.run else 'N/A'})")
    print(f"Target Device: {DEVICE}")

    # --- Run ---
    print("\nLoading model...")
    gan_model = load_quantized_gigagan(
        checkpoint_path=CHECKPOINT_TO_LOAD, device_str=DEVICE_STR,
        unconditional_gan=UNCONDITIONAL_GAN, image_size=IMAGE_SIZE,
        input_image_size=INPUT_IMAGE_SIZE
    )
    
    print("\nStarting image generation and profiling...")
    generate_images_and_profile(
        gan_model, num_images=NUM_IMAGES_TO_GENERATE, output_dir=OUTPUT_IMAGE_DIR,
        profile_log_dir=PROFILE_LOG_DIR, export_chrome_trace=EXPORT_CHROME_TRACE_FILE
    )

except ImportError as e:
    print(f"ImportError: {e}")
    if wandb.run: wandb.log({"error_type": "ImportError", "error_message": str(e)})
except Exception as e:
    tb_str = traceback.format_exc()
    print(f"An unexpected error occurred: {e}")
    print("Traceback:"); print(tb_str)
    if wandb.run: wandb.log({"error_type": str(type(e).__name__), "error_message": str(e), "traceback": tb_str})
finally:
    if wandb.run: # Ensure wandb.run exists before trying to finish
        wandb.finish() 
        print("Wandb run finished.")
    else:
        print("Wandb run was not initialized or failed to initialize.")

Wandb run initialized: GigaGAN_QProf_in32_out64_cuda_13mz (ID: rtt24nj3)
Target Device: cuda

Loading model...
Initializing GigaGANQuantizedV100 on device: cuda
Setting D multiscale_input_resolutions to: [32]


Generator: 43.71M
Discriminator: 30.74M


Model G device after .to(): cuda:0


/opt/conda/lib/python3.11/site-packages/gigagan_pytorch/gigagan_pytorch.py:1968: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pkg = torch.load(str(path))


Successfully called gan.load(./model-32.ckpt)
Starting quantization for V100 GPU...
Generator 'G' target device for quantization: cuda
Generator 'G' current device: cuda:0
Model G converted to FP16.
Critical components preservation step complete.
No eligible linear layers found or converted to INT8.
Selective INT8 quantization step complete.
Stability hooks step complete.
Quantization complete!
Model is in evaluation mode.

Starting image generation and profiling...
Generating images on device: cuda
Model type: Upsampler/Conditional. Dummy input size: (64, 64).
Dummy input tensor: shape torch.Size([1, 3, 64, 64]), dtype torch.float16

Starting profiling session (wait=1, warmup=1, active=3)...


/tmp/ipykernel_551/4236201076.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
/tmp/ipykernel_551/4236201076.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
[W


Finished generating 5 images (local saving commented out during profiling).

--- PyTorch Profiler Results (logged to wandb) ---
No significant CUDA activity in profiler, sorting by CPU time for console.
Error creating/logging wandb.Table for profiler: 'FunctionEventAvg' object has no attribute 'self_cpu_time_percent'
Console Profiler Table (sorted by self_cpu_time_total):
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  Total MFLOPs  
-------------------------------------------------------  ------------  ------------  -

Wandb run finished.
